# 정준상관분석 : 예시 (1)

## 자료 얻어내기

In [29]:
import numpy as np
import pandas as pd
from scipy.stats import wishart
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import CCA
from statsmodels.multivariate.cancorr import CanCorr
import matplotlib.pyplot as plt


In [30]:
# 시드값 설정
np.random.seed(2022)

# 자료 만들기
pp = 3
Sigma = wishart.rvs(df=pp, scale=np.identity(pp), size=1)
R1 = np.diag(1 / np.sqrt(np.diag(Sigma))) @ Sigma @ np.diag(1 / np.sqrt(np.diag(Sigma)))
Sigma = wishart.rvs(df=pp, scale=np.identity(pp), size=1)
R2 = np.diag(1 / np.sqrt(np.diag(Sigma))) @ Sigma @ np.diag(1 / np.sqrt(np.diag(Sigma)))

In [31]:
print("R1:")
print(pd.DataFrame(np.round(R1, 2)))
print("\nR2:")
print(pd.DataFrame(np.round(R2, 2)))

R1:
      0     1     2
0  1.00 -0.00 -0.15
1 -0.00  1.00 -0.08
2 -0.15 -0.08  1.00

R2:
      0     1     2
0  1.00  0.26  0.30
1  0.26  1.00  0.01
2  0.30  0.01  1.00


In [32]:
# 자료 뽑기 & 상관성 확인
nn = 10000
ss = np.block([[R1, np.zeros_like(R1)], [np.zeros_like(R2), R2]])  # 자료 합치기
pp2 = ss.shape[0]
mu = np.zeros(pp2)
d1 = np.random.multivariate_normal(mu, ss, nn)
print("Correlation matrix of d1:")
print(pd.DataFrame(np.round(np.corrcoef(d1, rowvar=False), 2)))

Correlation matrix of d1:
      0     1     2     3     4     5
0  1.00 -0.00 -0.17 -0.01 -0.00 -0.01
1 -0.00  1.00 -0.08  0.01  0.00  0.01
2 -0.17 -0.08  1.00  0.01 -0.01  0.01
3 -0.01  0.01  0.01  1.00  0.27  0.30
4 -0.00  0.00 -0.01  0.27  1.00  0.02
5 -0.01  0.01  0.01  0.30  0.02  1.00


In [33]:
# 행과 열을 교체한 새 자료
d1 = d1[:, np.array([i for sub in zip(range(pp2//2), range(pp2//2, pp2)) for i in sub])]
d1 = pd.DataFrame(d1, columns=[f'X{i+1}' for i in range(pp2)])
print("Reordered correlation matrix of d1:")
print(pd.DataFrame(np.round(d1.corr(), 2)))

Reordered correlation matrix of d1:
      X1    X2    X3    X4    X5    X6
X1  1.00 -0.01 -0.00 -0.00 -0.17 -0.01
X2 -0.01  1.00  0.01  0.27  0.01  0.30
X3 -0.00  0.01  1.00  0.00 -0.08  0.01
X4 -0.00  0.27  0.00  1.00 -0.01  0.02
X5 -0.17  0.01 -0.08 -0.01  1.00  0.01
X6 -0.01  0.30  0.01  0.02  0.01  1.00


In [34]:
# 행렬 간 상관관계
X = d1.iloc[:, :3]
Y = d1.iloc[:, 3:]
print("Matrix correlation between X and Y:")
print(np.round(X.corrwith(Y), 2))

Matrix correlation between X and Y:
X1   NaN
X2   NaN
X3   NaN
X4   NaN
X5   NaN
X6   NaN
dtype: float64


## 정준상관분석 수행

In [35]:
cca = CCA(n_components=3)
cca.fit(X, Y)
X_c, Y_c = cca.transform(X, Y)

print("Canonical correlations:")
canonical_corr = [np.corrcoef(X_c[:, i], Y_c[:, i])[0, 1] for i in range(3)]
print(np.round(canonical_corr, 2))

Canonical correlations:
[0.39 0.19 0.01]


## 정준변수점수 도출

In [36]:
score = X_c
print("Mean of canonical scores:")
print(np.round(np.mean(score, axis=0), 2))
print("Standard deviation of canonical scores:")
print(np.round(np.std(score, axis=0), 2))
print("Covariance of canonical scores:")
print(np.round(np.cov(score.T), 2))

Mean of canonical scores:
[ 0. -0.  0.]
Standard deviation of canonical scores:
[1. 1. 1.]
Covariance of canonical scores:
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [ 0. -0.  1.]]


## 적정 수의 정준변수쌍 산정

In [37]:
# 적정한 정준변수쌍 적정 수 산정
def wilks_lambda(canonical_corr):
    wilks_stats = (1 - np.array(canonical_corr) ** 2)
    wilks_lambda_stat = np.prod(wilks_stats)
    return wilks_lambda_stat

wilks_stat = wilks_lambda(canonical_corr)
print("Wilks' lambda statistic:", wilks_stat)


Wilks' lambda statistic: 0.8150953636783438


## 공헌도와 근사도

In [38]:
# 공헌도와 근사도
canonical_correlations = np.array(canonical_corr)
w = canonical_correlations**2
print("Contribution (squared canonical correlations):")
print(w)

Contribution (squared canonical correlations):
[1.55681797e-01 3.45500840e-02 6.32701976e-05]


In [39]:
print("Cumulative contribution (%):")
print(np.cumsum(w) / np.sum(w) * 100)

Cumulative contribution (%):
[ 81.81070091  99.96675155 100.        ]


# 패키지 없이 구현하기

In [40]:
ss = np.cov(d1, rowvar=False)
Sxx = ss[:3, :3]
Sxy = ss[:3, 3:]
Syy = ss[3:, 3:]

In [41]:
def sqrtm_inv(mat):
    eigvals, eigvecs = np.linalg.eigh(mat)
    return eigvecs @ np.diag(1/np.sqrt(eigvals)) @ eigvecs.T

Sxx_inv_sqrt = sqrtm_inv(Sxx)
Syy_inv_sqrt = sqrtm_inv(Syy)

In [42]:
# 연산1
tmp = Sxx_inv_sqrt @ Sxy @ np.linalg.inv(Syy) @ Sxy.T @ Sxx_inv_sqrt
eigvals, eigvecs = np.linalg.eigh(tmp)
P = eigvecs
u = Sxx_inv_sqrt @ P

# 연산2
tmp = Syy_inv_sqrt @ Sxy.T @ np.linalg.inv(Sxx) @ Sxy @ Syy_inv_sqrt
eigvals, eigvecs = np.linalg.eigh(tmp)
Q = eigvecs
v = Syy_inv_sqrt @ Q

In [43]:
# 라이브러리와 비교
print("Canonical coefficients from library (X):")
print(np.round(model.y_cancoef, 4))
print("Computed canonical coefficients (X):")
print(np.round(u, 4))

Canonical coefficients from library (X):
[[ 0.0004  0.0092 -0.0041]
 [-0.01    0.0004 -0.0002]
 [ 0.0001  0.0041  0.0091]]
Computed canonical coefficients (X):
[[ 0.4065  0.9167 -0.0374]
 [ 0.0226  0.0392  1.0003]
 [-0.9133  0.4096 -0.0096]]


In [44]:
print("Canonical coefficients from library (Y):")
print(np.round(model.x_cancoef, 4))
print("Computed canonical coefficients (Y):")
print(np.round(v, 4))

Canonical coefficients from library (Y):
[[-0.0066  0.0004 -0.0075]
 [-0.0004 -0.0099 -0.0004]
 [-0.0073  0.0001  0.0067]]
Computed canonical coefficients (Y):
[[-0.7492  0.0414 -0.6627]
 [-0.0363 -0.9859 -0.0426]
 [ 0.6684  0.0122 -0.7316]]
